In [1]:
import sys

sys.path.insert(0, "../..")
from pathlib import Path
from src.data import dataloader
from src.data import make_dataset

2022-06-14 16:37:31.430709: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-14 16:37:31.430753: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
eegPath = make_dataset.get_eeg()

2022-06-14 16:37:34.173 | INFO     | src.data.make_dataset:get_eeg:18 - Data is downloaded to ../../data/raw/datasets/eeg.


In [3]:
dataset = dataloader.BaseDataset(datapath = eegPath) #make dataset
print(len(dataset))

24


In [4]:
dataset.__getitem__(0)[0].shape #get shape of 1st item 

torch.Size([188, 14])

First the BaseDataStreamer is used (this is level one). This iterator automatically changes the window size if its larger than the smallest chunck in BaseDataSet. It also takes another chunck if the chunck does not have enough lines. For these reasons we always have the same shape tensors in one batch. The downside is that we throw away information by doing this.

In [5]:
loader = dataloader.BaseDataStreamer(dataset = dataset, window_size=10, batchsize=32).stream()
x,y = next(loader)
x.shape, y.shape


(torch.Size([32, 10, 14]), torch.Size([32]))

If trying to use a window size of 21, the BaseDataIterator will tell and set to 21:

In [6]:
loader = dataloader.BaseDataStreamer(dataset = dataset, window_size=25, batchsize=32).stream()
x,y = next(loader)

Maximum window length is 21, setting window length to 21.
                Use PaddedDataStreamer for bigger window size


In [7]:
x

tensor([[[4261.0298, 3984.6201, 4246.1499,  ..., 4269.7402, 4582.0498,
          4312.8198],
         [4257.4399, 3988.2100, 4240.5098,  ..., 4260.0000, 4573.3301,
          4306.1499],
         [4252.3101, 3984.6201, 4230.7700,  ..., 4257.9502, 4570.2598,
          4309.2300],
         ...,
         [4269.7402, 3986.6699, 4253.3301,  ..., 4275.3799, 4573.3301,
          4321.5400],
         [4269.7402, 3986.1499, 4250.7700,  ..., 4274.3599, 4577.9502,
          4320.5098],
         [4272.3101, 3992.3101, 4249.7402,  ..., 4269.2300, 4574.3599,
          4320.0000]],

        [[4409.7402, 4111.2798, 4290.7700,  ..., 4296.9199, 4632.8198,
          4463.0801],
         [4432.3101, 4116.4102, 4297.9502,  ..., 4300.0000, 4646.1499,
          4474.8701],
         [4439.4902, 4124.6201, 4296.9199,  ..., 4292.8198, 4643.0801,
          4469.7402],
         ...,
         [4385.1299, 4089.2300, 4279.4902,  ..., 4288.7202, 4627.1802,
          4422.0498],
         [4374.3599, 4083.0801, 4280.000

Another way of using the data is padding the data if the window size is too big (this is level 2). In the PaddedDataIterator the data is padded if the sequence is too short. For this reason we can use a larger window size than the size of the shortest observation. The padded data iterator also has an extra argument, min_nr_lines which specifies how many items there should at least be in an observation, if this condition does not equal to true, the previous window will be taken. 

In [13]:
loaderPadded = dataloader.PaddedDataStreamer(dataset = dataset, window_size=40, batchsize=32, min_nr_lines= 5).stream()
x,y = next(loaderPadded)
x,y = next(loaderPadded)
x,y = next(loaderPadded)
x,y = next(loaderPadded)
x,y = next(loaderPadded)
x,y = next(loaderPadded)

x.shape, y.shape

(torch.Size([32, 40, 14]), torch.Size([32]))

In [10]:
x

tensor([[[4307.6899, 4021.0300, 4263.0801,  ..., 4274.3599, 4607.6899,
          4361.0298],
         [4309.7402, 4026.6699, 4265.1299,  ..., 4277.4399, 4614.3599,
          4363.5898],
         [4300.5098, 4025.1299, 4260.5098,  ..., 4272.8198, 4609.7402,
          4352.8198],
         ...,
         [4306.6699, 4016.9199, 4266.1499,  ..., 4284.1001, 4620.5098,
          4382.0498],
         [4294.8701, 4007.1799, 4260.0000,  ..., 4280.5098, 4609.2300,
          4368.7202],
         [4290.2598, 4001.5400, 4250.7700,  ..., 4278.4600, 4598.4600,
          4351.2798]],

        [[4332.8198, 4018.9700, 4260.0000,  ..., 4281.0298, 4612.8198,
          4387.6899],
         [4340.0000, 4022.5601, 4261.0298,  ..., 4282.5601, 4615.8999,
          4394.8701],
         [4341.5400, 4027.6899, 4259.4902,  ..., 4279.4902, 4617.9502,
          4398.4600],
         ...,
         [4420.5098, 4068.7200, 4293.8501,  ..., 4315.3799, 4675.3799,
          4460.5098],
         [4417.4399, 4069.2300, 4291.279